In [1]:
import tensorflow as tf
import numpy as np

from model import DispNet
from dataset import Scene_Flow_disparity

import os
import cv2
import time
import datetime
from random import shuffle
from random import randrange
from tqdm import tqdm_notebook

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
QUEUE_LENGTH = 20
NUM_OF_THREAD = 12

epoch = 30
rate_of_paths = 1.0
train_batch_size = 2
learning_rate = 0.1**4

In [3]:
def DataGenerator(data_paths, batch_size = 1):
    train_left_inputBatch = []
    train_right_inputBatch = []
    train_left_groundTruthBatch = []
    for dir_ in data_paths:
        l_img_nparray = cv2.imread(dir_, cv2.IMREAD_COLOR)  
        r_img_nparray = cv2.imread(dir_.replace("./l", "./r"), cv2.IMREAD_COLOR) 
        train_left_inputBatch += [l_img_nparray]
        train_right_inputBatch += [r_img_nparray]
        if len(train_left_inputBatch) == batch_size:
            batch_input_x_l = np.array(train_left_inputBatch).astype(np.float32)
            batch_input_x_r = np.array(train_right_inputBatch).astype(np.float32)
            train_left_inputBatch = []
            train_right_inputBatch = []
            yield (batch_input_x_l, batch_input_x_r)

In [4]:
prediction_paths = ['./test_set/CG_Testset1_20180711/Image_00.png', 
                    './test_set/ETRI_chef_0_100/Image_00.png',
                    './test_set/middleburry/Image_00.png',
                    './test_set/Sceneflow/Image_00.png',
                    './test_set/TechnicolorPainter_pr_100_#00_#15/Image_00.png']
prediction_generator = DataGenerator(data_paths = prediction_paths, batch_size = 1)

model = DispNet(img_height = 512,
                img_width = 960,
                img_depth = 3,
                learning_rate = learning_rate, 
                batch_size = 1,
                mode = 'correlation_')

queue = tf.FIFOQueue(QUEUE_LENGTH, dtypes=[tf.float32, tf.float32], shapes=[[1, 512, 960, 3], 
                                                                            [1, 512, 960, 3]])
with tf.device('/cpu:0'):
    enqueue = queue.enqueue((model.left_image, model.right_image))
    qr = tf.train.QueueRunner(queue, [enqueue]*NUM_OF_THREAD)
    sess = tf.Session()
    coord = tf.train.Coordinator()
    threads = qr.create_threads(sess, coord=coord, start=True)
    X_L_batch, X_R_batch = queue.dequeue()
with sess.as_default():
    train_op, loss, prediction, tensorboard = model.inference(X_L_batch, X_R_batch, model.ground_truth)
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    saver = tf.train.Saver()
    tf.reset_default_graph()
    ckpt_path = saver.restore(sess, "./checkpoints/tf-DispNet/trained_in_SceneFlowDataset/batch_size_2-{0}epoch".format(9))
    
    print()
    for step in tqdm_notebook(range(0, (int)(len(prediction_paths) / 1))):
        net_input = list(next(prediction_generator))
        pre = sess.run([prediction], feed_dict={X_L_batch: net_input[0], 
                                                X_R_batch: net_input[1]})
        if step % 1 == 0:
            cv2.imwrite('%d.png' % (step+1), pre[0][0,:].astype(np.uint8))

input image resized by (height = 512, width = 960)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, You must feed a value for placeholder tensor 'right_image' with dtype float and shape [1,512,960,3]
	 [[Node: right_image = Placeholder[dtype=DT_FLOAT, shape=[1,512,960,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
INFO:tensorflow:Restoring parameters from ./checkpoints/tf-DispNet/trained_in_SceneFlowDataset/batch_size_2-9epoch



HBox(children=(IntProgress(value=0, max=5), HTML(value='')))